# Дипломчик
### Коды Гоппы

In [1]:
from sage.all import *

In [36]:
class utils:
    def random_permutation_matrix(n):
        m = identity_matrix(n,n)
        # todo: perform random swaps of rows and cols
        return m


In [55]:
F = GF(2^4, 'a')

for i,x in enumerate(F):  print("{} {}".format(i, x))

R.<x> = F[]
RR.<a> = F
g = x^2 + x + a^3
L = [a for a in F.list() if g(a) != 0]
C = codes.GoppaCode(g, L)

print(C.parity_check_matrix())
print('\n')
print(C.generator_matrix())

G = C.generator_matrix()

r = len(G.rows())
c = len(G.columns())

S = random_matrix(GF(2), r,r, algorithm='unimodular')
print(S)

P = utils.random_permutation_matrix(c)
print(P)

GG = S * G * P
print(GG)

0 0
1 a
2 a^2
3 a^3
4 a + 1
5 a^2 + a
6 a^3 + a^2
7 a^3 + a + 1
8 a^2 + 1
9 a^3 + a
10 a^2 + a + 1
11 a^3 + a^2 + a
12 a^3 + a^2 + a + 1
13 a^3 + a^2 + 1
14 a^3 + 1
15 1
[1 1 0 0 1 0 1 0 0 0 0 0 0 1 0 1]
[1 1 0 1 1 1 0 0 0 0 1 0 0 0 1 1]
[1 0 0 0 0 0 1 1 0 1 0 1 1 1 0 1]
[1 0 1 1 0 0 0 1 1 1 0 0 0 0 1 1]
[0 0 0 1 1 0 1 1 0 1 0 1 1 0 1 1]
[0 1 1 1 0 0 0 0 1 0 1 0 0 0 0 1]
[0 1 1 1 1 1 0 1 1 0 1 1 0 1 1 1]
[0 0 0 1 0 1 1 1 1 0 1 1 1 1 0 1]


[1 0 0 0 0 0 0 1 0 0 1 0 1 0 1 1]
[0 1 0 0 0 0 0 1 0 0 0 1 1 0 0 1]
[0 0 1 0 0 0 0 1 0 0 0 1 0 1 1 1]
[0 0 0 1 0 0 0 1 0 1 0 0 0 1 0 1]
[0 0 0 0 1 0 0 1 0 1 1 1 0 0 1 1]
[0 0 0 0 0 1 0 0 0 0 1 1 1 1 1 1]
[0 0 0 0 0 0 1 0 0 1 1 0 1 0 0 1]
[0 0 0 0 0 0 0 0 1 1 0 0 1 1 1 1]
[1 0 0 1 0 1 1 0]
[1 1 0 1 1 0 0 1]
[0 0 1 1 0 0 1 0]
[1 1 0 0 0 1 1 0]
[0 0 0 1 0 1 1 0]
[0 0 0 1 0 0 1 0]
[1 1 0 0 1 1 0 1]
[0 0 1 1 0 0 1 1]
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0